# Building an ITSG-33 RAG chatbot with LangChain, Ollama, and Neo4j

In [50]:
%%sh
pip install langchain langchain-community langchain-experimental json-repair pypdf neo4j

In [1]:
import boto3, json
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain.docstore.document import Document


## Connect to Ollama 

In [2]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="llama3:latest",
    base_url='http://ollama-server-2:11434',
    temperature=0,
    top_p=0,
    top_k=40
)  

llm.invoke("Tell me a joke")

"Here's one:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!\n\nHope that made you smile! Do you want to hear another one?"

In [3]:
llm_graph_transformer = LLMGraphTransformer(llm=llm)

## Security Control Catalogue
# Nodes: Control, Control Enhancement, Family, Class, Document, Profile, SuggestedPriority, FunctionalGroup
# Relationships: IS_PART_OF, BLONGS_TO, REFERENCES, IS_SUGGESTED_FOR, ENHANCES

In [58]:
from langchain.document_loaders import PyPDFLoader

pdfUri = '../Downloads/itsg33-ann1-eng.pdf'
loader = PyPDFLoader(pdfUri)
documents = loader.load()

In [60]:
%%time

graph_documents = llm_graph_transformer.convert_to_graph_documents(documents)

CPU times: user 2.84 s, sys: 357 ms, total: 3.2 s
Wall time: 7min 6s


In [61]:
for node in graph_documents[0].nodes:
    print(node)

id='Departmental IT Security Risk' type='Concept'
id='A Lifecycle Approach' type='Concept'
id='Management Activities' type='Concept'
id='IT Security Risk Management' type='Concept'


In [63]:
for relationship in graph_documents[1].relationships:
    print(relationship)

source=Node(id='Communications Security Establishment Canada (CSE C)', type='Organization') target=Node(id='ITSG -33', type='Publication') type='ISSUED_UNDER_AUTHORITY_OF'
source=Node(id='Toni Moffa', type='Person') target=Node(id='This publication', type='Publication') type='SIGNED_BY'
source=Node(id='IT Security Client Services Representative at CSE C', type='Role') target=Node(id='Suggestions for amendments', type='Request') type='FORWARDED_TO'
source=Node(id='IT Security Client Services area at CSE C', type='Role') target=Node(id='itsclientservices@cse-cst.gc.ca', type='Email') type='CONTACTED_BY_EMAIL'
source=Node(id='IT Security Client Services area at CSE C', type='Role') target=Node(id='(613) 991-7654', type='Phone Number') type='CONTACTED_BY_PHONE'


In [64]:
len(graph_documents)

56

In [65]:
%%time
NEO4J_URI = 'bolt://localhost:7687'
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = 'neo4jgenai'
NEO4J_DATABASE = 'neo4j'

kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

CPU times: user 101 ms, sys: 25.3 ms, total: 126 ms
Wall time: 574 ms


In [66]:
%%time
kg.add_graph_documents(graph_documents)

CPU times: user 111 ms, sys: 11.3 ms, total: 123 ms
Wall time: 10.9 s
